In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')
test_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 133) (3492890, 132)
CPU times: user 3min 18s, sys: 11.7 s, total: 3min 30s
Wall time: 40.9 s


In [5]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 4min 2s, sys: 13.9 s, total: 4min 16s
Wall time: 51.8 s


In [6]:
%%time
test_metadata_kaggle = test_metadata_kaggle.reset_index(drop=True)
test_metadata = test_metadata.reset_index(drop=True)

CPU times: user 6.21 s, sys: 9.57 s, total: 15.8 s
Wall time: 15.9 s


In [7]:
print(train_metadata_kaggle['object_id'].equals(train_metadata['object_id']))

True


In [8]:
print(test_metadata_kaggle['object_id'].equals(test_metadata['object_id']))

True


In [9]:
temp_columns = ['object_id','A0_max_flux','A0_min_flux','A0_mean_flux','A0_median_flux','A0_std_flux',
'A1_max_flux','A1_min_flux','A1_mean_flux','A1_median_flux','A1_std_flux',
'A2_max_flux','A2_min_flux','A2_mean_flux','A2_median_flux','A2_std_flux',
'A3_max_flux','A3_min_flux','A3_mean_flux','A3_median_flux','A3_std_flux',
'A4_max_flux','A4_min_flux','A4_mean_flux','A4_median_flux','A4_std_flux',
'A5_max_flux','A5_min_flux','A5_mean_flux','A5_median_flux','A5_std_flux']

In [10]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [11]:
train_metadata.drop([x for x in train_metadata.columns if x not in temp_columns], axis=1,inplace=True)
gc.collect()

525

In [12]:
test_metadata.drop([x for x in test_metadata.columns if x not in temp_columns], axis=1,inplace=True)
gc.collect()

7

In [13]:
print(train_metadata.shape,test_metadata.shape)

(7848, 31) (3492890, 31)


In [14]:
test_metadata_kaggle.tail()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux
3492885,130787966,-49.490440,75.235832,3.905430,1.703813,21.009541,0.625654,1.341227,70.066055,14.846084,10.951475,12.628468,1.307980,0.013793,374.420803,8.678639,5.954086e+03,7.560569,15.902124,124.726272,31.936629,7.843372,35.655183,32.200453,0.154555,-0.174445,23.565507,13.595040,6.803259,2.380728,63.739853,39.279845,3.023876,1.647382,82.545904,81.181195,0.670890,0.882368,187.846479,74.176357,3.792912,1.576256,172.934431,184.079710,-0.297825,0.291885,145.0,8.0,10.0,20.061909,0.151706,4.0,43.0,89.761400,89.761400,89.7614,0.4493,0.9954,41.9836,0.036,1.937777,2.277911,1.215719,35.655180,2.377012,0.0000,0.0909,0.0,0.0,0.000,0.00000,0.5557,-2.620740,0.0,5.474970e-13,0.103244,2.16581,0.050748,0.683935,2.26490,0.829141,2.439200,2.16581,0.500000,0.953504,0.808853,0.966782,2.32229,0.185745,19.47180,0.427032,10.79380,0.000000,1.000000,1.000000,0,0,0,0,-20.260712,72.660479,-92.906480,71.556374,28.116876,-6.290332,4.964574,2.744162,46.834716,-14.615577,46.582108,56.130652,50.831205,-0.426830,4.249098,-5.116519,55.883266,25.372714,7.267148,48.022585,53.139104,-14.781784,42.

In [15]:
test_metadata.tail()

,object_id,A0_min_flux,A0_max_flux,A0_std_flux,A0_mean_flux,A0_median_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_mean_flux,A1_median_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_mean_flux,A2_median_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_mean_flux,A3_median_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_mean_flux,A4_median_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_mean_flux,A5_median_flux
3492885,130787966,-19.231127,18.327864,10.015860,2.377012,0.194802,-5.105165,22.772732,7.467904,2.143,0.8790,-2.620740,18.966825,5.440,3.541103,2.189612,-8.542406,24.314121,8.09,3.588952,1.232520,-23.242554,73.366840,19.793636,6.059564,3.450523,-49.490440,75.235830,32.195312,3.679456,2.575351
3492886,130787971,-13.702611,5.737575,6.879981,-3.376497,-5.786102,-6.108143,14.889024,5.566116,2.176,1.1770,-13.268991,44.679916,13.820,4.632854,1.252785,-11.002852,9.428670,6.11,0.622537,0.740433,-27.026964,59.105938,17.087936,6.418810,4.378683,-58.301780,46.180240,28.761728,2.859332,10.149758
3492887,130787974,-19.901522,117.104140,33.753580,11.206562,5.168170,-3.021157,65.796760,25.748106,14.120,2.7200,-4.419514,331.153630,117.700,79.828990,5.408035,-7.961402,308.078770,92.80,54.059563,3.760870,-15.829834,341.318900,122.374030,64.385560,4.701163,-79.991745,307.034970,90.453160,26.689178,-4.267330
3492888,130788053,-9.348281,72.267590,23.513706,6.616929,0.730023,-1.932591,66.417595,24.979921,9.860,0.5176,-10.273894,5.598794,3.746,-0.260037,-0.425032,-11.894145,17.121632,7.68,1.022865,1.275511,-29.653970,17.978836,11.335240,0.889137,0.173198,-31.093233,47.343845,17.408049,4.865474,2.143283
3492889,130788054,-13.072737,30.335392,11.523481,0.690040,-1.810264,-3.566863,47.519875,12.820799,3.791,0.1388,-5.267357,27.842110,7.820,2.587629,0.787241,-6.765305,25.873100,9.10,4.772007,2.447138,-12.543804,24.374737,9.450494,5.907181,4.280222,-29.865211,92.339670,32.776836,17.445766,6.951662


In [16]:
train_metadata_kaggle.tail()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux
7843,130739978,-105.375282,517.602478,6.786007,1.599034,50.600944,7.259594,1.700991,72.230759,16.391897,11.132936,14.645405,1.401451,0.034014,3178.336379,11.992239,4.882531e+05,11.679274,153.619074,622.977760,91.803294,4.055341,21.873135,28.266883,5.705815,1.784807,162.456175,162.296936,13.667479,3.680811,35.689574,7.912341,8.977344,2.753125,117.413905,65.508160,-0.460370,0.410709,42.460987,115.925375,0.845991,0.431793,702.571174,302.817480,20.652438,3.998852,147.0,4.0,15.0,32.225018,0.053633,1.0,97.0,199.953250,199.953250,799.8130,0.0000,0.0000,NaN,0.013,65,1.568287,3.671722,0.000000,21.873135,1.286655,0.2426,0.07140,2,0.0000,0.1691,0.02856,0.6250,-7.383,1,2.351330e-29,1.100730e-01,3.12249,1.669840,0.413225,2.63550,0.698985,11.699300,3.12249,0.666667,0.961661,0.881326,0.257244,5.264420,1.573640,16.47190,0.527594,22.95970,0.000,1.0,1.0,0,0,0,0,-114.710914,506.473179,-272.734698,497.529038,541.19002,-115.000845,6.9940,458.076275,26.931324,-17.919780,28.676722,460.481299,478.301014,0.227830,449.624292,-117.699845,466.670374,83.113745,9.356076,-109.105746,8.59

In [17]:
train_metadata.head()

,object_id,A0_min_flux,A0_max_flux,A0_std_flux,A0_mean_flux,A0_median_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_mean_flux,A1_median_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_mean_flux,A2_median_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_mean_flux,A3_median_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_mean_flux,A4_median_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_mean_flux,A5_median_flux
0,615,-116.913220,125.182810,83.944730,-3.254554,-10.015225,-1100.000,660.626340,601.787300,-385.8000,-488.0000,-682.000,611.984560,455.121340,-134.146560,-265.8000,-530.500,445.737060,335.425050,-121.103500,-162.1000,-422.184500,381.953740,291.803440,-55.954594,-103.541370,-422.815100,378.188140,294.779500,-47.449850,-85.524310
1,713,-14.735178,14.509829,7.113509,-2.720398,-3.096805,-11.720,9.129021,5.712334,-1.0200,-0.5615,-10.070,10.529041,5.770738,-0.794238,-0.1180,-12.400,11.330316,6.450413,-0.986966,-0.0739,-12.286801,9.827934,6.406989,-0.900261,-0.792176,-14.211164,14.770886,7.094073,-1.794175,-2.463012
2,730,-3.459960,5.942166,1.828872,-0.048080,0.024093,-3.393,5.693109,1.807229,0.1411,0.1714,-2.850,20.994710,5.559483,2.400870,0.4917,-5.438,33.572100,8.191987,3.236164,0.6606,-5.836310,41.159980,10.710344,4.308728,1.004354,-19.159811,47.310060,13.332758,4.539396,2.542647
3,745,-3.874349,18.014029,4.374445,1.797523,1.056714,-3.620,192.244300,25.964659,5.7200,0.8880,-2.160,220.795210,31.957998,9.711532,0.4243,-4.945,203.250700,34.967697,14.412925,1.3610,-15.494463,183.633120,33.069054,13.134436,1.270150,-10.249387,141.513290,26.060130,10.746138,2.749555
4,1124,-6.804703,5.330927,2.360085,0.660948,0.581027,-2.623,37.170177,8.107525,4.6330,1.1540,-2.084,106.671690,21.319853,10.243968,0.8890,-2.800,139.818400,26.270649,11.086555,1.0140,-16.543753,143.600190,26.865913,9.906102,1.745012,-10.860540,109.157585,21.434628,6.896741,1.973272


In [18]:
######################################TRAIN####################################################3
#A0
train_metadata['A0_max_min_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_max_mean_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_max_median_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_median_flux']
train_metadata['A0_median_mean_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_median_min_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_mean_min_diff_flux'] = train_metadata['A0_mean_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_minus_3_sigma'] = train_metadata['A0_mean_flux'] - 3*train_metadata['A0_std_flux']
train_metadata['A0_minus_2_sigma'] = train_metadata['A0_mean_flux'] - 2*train_metadata['A0_std_flux']
train_metadata['A0_minus_1_sigma'] = train_metadata['A0_mean_flux'] - 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_1_sigma'] = train_metadata['A0_mean_flux'] + 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_2_sigma'] = train_metadata['A0_mean_flux'] + 2*train_metadata['A0_std_flux']
train_metadata['A0_plus_3_sigma'] = train_metadata['A0_mean_flux'] + 3*train_metadata['A0_std_flux']
#A1
train_metadata['A1_max_min_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_max_mean_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_max_median_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_median_flux']
train_metadata['A1_median_mean_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_median_min_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_mean_min_diff_flux'] = train_metadata['A1_mean_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_minus_3_sigma'] = train_metadata['A1_mean_flux'] - 3*train_metadata['A1_std_flux']
train_metadata['A1_minus_2_sigma'] = train_metadata['A1_mean_flux'] - 2*train_metadata['A1_std_flux']
train_metadata['A1_minus_1_sigma'] = train_metadata['A1_mean_flux'] - 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_1_sigma'] = train_metadata['A1_mean_flux'] + 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_2_sigma'] = train_metadata['A1_mean_flux'] + 2*train_metadata['A1_std_flux']
train_metadata['A1_plus_3_sigma'] = train_metadata['A1_mean_flux'] + 3*train_metadata['A1_std_flux']
#A2
train_metadata['A2_max_min_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_max_mean_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_max_median_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_median_flux']
train_metadata['A2_median_mean_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_median_min_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_mean_min_diff_flux'] = train_metadata['A2_mean_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_minus_3_sigma'] = train_metadata['A2_mean_flux'] - 3*train_metadata['A2_std_flux']
train_metadata['A2_minus_2_sigma'] = train_metadata['A2_mean_flux'] - 2*train_metadata['A2_std_flux']
train_metadata['A2_minus_1_sigma'] = train_metadata['A2_mean_flux'] - 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_1_sigma'] = train_metadata['A2_mean_flux'] + 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_2_sigma'] = train_metadata['A2_mean_flux'] + 2*train_metadata['A2_std_flux']
train_metadata['A2_plus_3_sigma'] = train_metadata['A2_mean_flux'] + 3*train_metadata['A2_std_flux']
#A3
train_metadata['A3_max_min_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_max_mean_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_max_median_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_median_flux']
train_metadata['A3_median_mean_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_median_min_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_mean_min_diff_flux'] = train_metadata['A3_mean_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_minus_3_sigma'] = train_metadata['A3_mean_flux'] - 3*train_metadata['A3_std_flux']
train_metadata['A3_minus_2_sigma'] = train_metadata['A3_mean_flux'] - 2*train_metadata['A3_std_flux']
train_metadata['A3_minus_1_sigma'] = train_metadata['A3_mean_flux'] - 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_1_sigma'] = train_metadata['A3_mean_flux'] + 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_2_sigma'] = train_metadata['A3_mean_flux'] + 2*train_metadata['A3_std_flux']
train_metadata['A3_plus_3_sigma'] = train_metadata['A3_mean_flux'] + 3*train_metadata['A3_std_flux']
#A4
train_metadata['A4_max_min_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_max_mean_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_max_median_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_median_flux']
train_metadata['A4_median_mean_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_median_min_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_mean_min_diff_flux'] = train_metadata['A4_mean_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_minus_3_sigma'] = train_metadata['A4_mean_flux'] - 3*train_metadata['A4_std_flux']
train_metadata['A4_minus_2_sigma'] = train_metadata['A4_mean_flux'] - 2*train_metadata['A4_std_flux']
train_metadata['A4_minus_1_sigma'] = train_metadata['A4_mean_flux'] - 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_1_sigma'] = train_metadata['A4_mean_flux'] + 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_2_sigma'] = train_metadata['A4_mean_flux'] + 2*train_metadata['A4_std_flux']
train_metadata['A4_plus_3_sigma'] = train_metadata['A4_mean_flux'] + 3*train_metadata['A4_std_flux']
#A5
train_metadata['A5_max_min_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_max_mean_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_max_median_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_median_flux']
train_metadata['A5_median_mean_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_median_min_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_mean_min_diff_flux'] = train_metadata['A5_mean_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_minus_3_sigma'] = train_metadata['A5_mean_flux'] - 3*train_metadata['A5_std_flux']
train_metadata['A5_minus_2_sigma'] = train_metadata['A5_mean_flux'] - 2*train_metadata['A5_std_flux']
train_metadata['A5_minus_1_sigma'] = train_metadata['A5_mean_flux'] - 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_1_sigma'] = train_metadata['A5_mean_flux'] + 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_2_sigma'] = train_metadata['A5_mean_flux'] + 2*train_metadata['A5_std_flux']
train_metadata['A5_plus_3_sigma'] = train_metadata['A5_mean_flux'] + 3*train_metadata['A5_std_flux']
#######################################TEST#########################################################
#A0
test_metadata['A0_max_min_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_max_mean_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_mean_flux']
test_metadata['A0_max_median_diff_flux'] = test_metadata['A0_max_flux'] - test_metadata['A0_median_flux']
test_metadata['A0_median_mean_diff_flux'] = test_metadata['A0_median_flux'] - test_metadata['A0_mean_flux']
test_metadata['A0_median_min_diff_flux'] = test_metadata['A0_median_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_mean_min_diff_flux'] = test_metadata['A0_mean_flux'] - test_metadata['A0_min_flux']
test_metadata['A0_minus_3_sigma'] = test_metadata['A0_mean_flux'] - 3*test_metadata['A0_std_flux']
test_metadata['A0_minus_2_sigma'] = test_metadata['A0_mean_flux'] - 2*test_metadata['A0_std_flux']
test_metadata['A0_minus_1_sigma'] = test_metadata['A0_mean_flux'] - 1*test_metadata['A0_std_flux']
test_metadata['A0_plus_1_sigma'] = test_metadata['A0_mean_flux'] + 1*test_metadata['A0_std_flux']
test_metadata['A0_plus_2_sigma'] = test_metadata['A0_mean_flux'] + 2*test_metadata['A0_std_flux']
test_metadata['A0_plus_3_sigma'] = test_metadata['A0_mean_flux'] + 3*test_metadata['A0_std_flux']
#A1
test_metadata['A1_max_min_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_max_mean_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_mean_flux']
test_metadata['A1_max_median_diff_flux'] = test_metadata['A1_max_flux'] - test_metadata['A1_median_flux']
test_metadata['A1_median_mean_diff_flux'] = test_metadata['A1_median_flux'] - test_metadata['A1_mean_flux']
test_metadata['A1_median_min_diff_flux'] = test_metadata['A1_median_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_mean_min_diff_flux'] = test_metadata['A1_mean_flux'] - test_metadata['A1_min_flux']
test_metadata['A1_minus_3_sigma'] = test_metadata['A1_mean_flux'] - 3*test_metadata['A1_std_flux']
test_metadata['A1_minus_2_sigma'] = test_metadata['A1_mean_flux'] - 2*test_metadata['A1_std_flux']
test_metadata['A1_minus_1_sigma'] = test_metadata['A1_mean_flux'] - 1*test_metadata['A1_std_flux']
test_metadata['A1_plus_1_sigma'] = test_metadata['A1_mean_flux'] + 1*test_metadata['A1_std_flux']
test_metadata['A1_plus_2_sigma'] = test_metadata['A1_mean_flux'] + 2*test_metadata['A1_std_flux']
test_metadata['A1_plus_3_sigma'] = test_metadata['A1_mean_flux'] + 3*test_metadata['A1_std_flux']
#A2
test_metadata['A2_max_min_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_max_mean_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_mean_flux']
test_metadata['A2_max_median_diff_flux'] = test_metadata['A2_max_flux'] - test_metadata['A2_median_flux']
test_metadata['A2_median_mean_diff_flux'] = test_metadata['A2_median_flux'] - test_metadata['A2_mean_flux']
test_metadata['A2_median_min_diff_flux'] = test_metadata['A2_median_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_mean_min_diff_flux'] = test_metadata['A2_mean_flux'] - test_metadata['A2_min_flux']
test_metadata['A2_minus_3_sigma'] = test_metadata['A2_mean_flux'] - 3*test_metadata['A2_std_flux']
test_metadata['A2_minus_2_sigma'] = test_metadata['A2_mean_flux'] - 2*test_metadata['A2_std_flux']
test_metadata['A2_minus_1_sigma'] = test_metadata['A2_mean_flux'] - 1*test_metadata['A2_std_flux']
test_metadata['A2_plus_1_sigma'] = test_metadata['A2_mean_flux'] + 1*test_metadata['A2_std_flux']
test_metadata['A2_plus_2_sigma'] = test_metadata['A2_mean_flux'] + 2*test_metadata['A2_std_flux']
test_metadata['A2_plus_3_sigma'] = test_metadata['A2_mean_flux'] + 3*test_metadata['A2_std_flux']
#A3
test_metadata['A3_max_min_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_max_mean_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_mean_flux']
test_metadata['A3_max_median_diff_flux'] = test_metadata['A3_max_flux'] - test_metadata['A3_median_flux']
test_metadata['A3_median_mean_diff_flux'] = test_metadata['A3_median_flux'] - test_metadata['A3_mean_flux']
test_metadata['A3_median_min_diff_flux'] = test_metadata['A3_median_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_mean_min_diff_flux'] = test_metadata['A3_mean_flux'] - test_metadata['A3_min_flux']
test_metadata['A3_minus_3_sigma'] = test_metadata['A3_mean_flux'] - 3*test_metadata['A3_std_flux']
test_metadata['A3_minus_2_sigma'] = test_metadata['A3_mean_flux'] - 2*test_metadata['A3_std_flux']
test_metadata['A3_minus_1_sigma'] = test_metadata['A3_mean_flux'] - 1*test_metadata['A3_std_flux']
test_metadata['A3_plus_1_sigma'] = test_metadata['A3_mean_flux'] + 1*test_metadata['A3_std_flux']
test_metadata['A3_plus_2_sigma'] = test_metadata['A3_mean_flux'] + 2*test_metadata['A3_std_flux']
test_metadata['A3_plus_3_sigma'] = test_metadata['A3_mean_flux'] + 3*test_metadata['A3_std_flux']
#A4
test_metadata['A4_max_min_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_max_mean_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_mean_flux']
test_metadata['A4_max_median_diff_flux'] = test_metadata['A4_max_flux'] - test_metadata['A4_median_flux']
test_metadata['A4_median_mean_diff_flux'] = test_metadata['A4_median_flux'] - test_metadata['A4_mean_flux']
test_metadata['A4_median_min_diff_flux'] = test_metadata['A4_median_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_mean_min_diff_flux'] = test_metadata['A4_mean_flux'] - test_metadata['A4_min_flux']
test_metadata['A4_minus_3_sigma'] = test_metadata['A4_mean_flux'] - 3*test_metadata['A4_std_flux']
test_metadata['A4_minus_2_sigma'] = test_metadata['A4_mean_flux'] - 2*test_metadata['A4_std_flux']
test_metadata['A4_minus_1_sigma'] = test_metadata['A4_mean_flux'] - 1*test_metadata['A4_std_flux']
test_metadata['A4_plus_1_sigma'] = test_metadata['A4_mean_flux'] + 1*test_metadata['A4_std_flux']
test_metadata['A4_plus_2_sigma'] = test_metadata['A4_mean_flux'] + 2*test_metadata['A4_std_flux']
test_metadata['A4_plus_3_sigma'] = test_metadata['A4_mean_flux'] + 3*test_metadata['A4_std_flux']
#A5
test_metadata['A5_max_min_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_max_mean_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_mean_flux']
test_metadata['A5_max_median_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_median_flux']
test_metadata['A5_median_mean_diff_flux'] = test_metadata['A5_median_flux'] - test_metadata['A5_mean_flux']
test_metadata['A5_median_min_diff_flux'] = test_metadata['A5_median_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_mean_min_diff_flux'] = test_metadata['A5_mean_flux'] - test_metadata['A5_min_flux']
test_metadata['A5_minus_3_sigma'] = test_metadata['A5_mean_flux'] - 3*test_metadata['A5_std_flux']
test_metadata['A5_minus_2_sigma'] = test_metadata['A5_mean_flux'] - 2*test_metadata['A5_std_flux']
test_metadata['A5_minus_1_sigma'] = test_metadata['A5_mean_flux'] - 1*test_metadata['A5_std_flux']
test_metadata['A5_plus_1_sigma'] = test_metadata['A5_mean_flux'] + 1*test_metadata['A5_std_flux']
test_metadata['A5_plus_2_sigma'] = test_metadata['A5_mean_flux'] + 2*test_metadata['A5_std_flux']
test_metadata['A5_plus_3_sigma'] = test_metadata['A5_mean_flux'] + 3*test_metadata['A5_std_flux']

In [19]:
#F1
train_metadata['div_A5_A4_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] / train_metadata['A4_max_min_diff_flux']
train_metadata['div_A5_A3_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] / train_metadata['A3_max_min_diff_flux']
train_metadata['div_A5_A2_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] / train_metadata['A2_max_min_diff_flux']
train_metadata['div_A5_A1_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] / train_metadata['A1_max_min_diff_flux']
train_metadata['div_A5_A0_max_min_flux'] = train_metadata['A5_max_min_diff_flux'] / train_metadata['A0_max_min_diff_flux']
train_metadata['div_A4_A3_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] / train_metadata['A3_max_min_diff_flux']
train_metadata['div_A4_A2_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] / train_metadata['A2_max_min_diff_flux']
train_metadata['div_A4_A1_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] / train_metadata['A1_max_min_diff_flux']
train_metadata['div_A4_A0_max_min_flux'] = train_metadata['A4_max_min_diff_flux'] / train_metadata['A0_max_min_diff_flux']
train_metadata['div_A3_A2_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] / train_metadata['A2_max_min_diff_flux']
train_metadata['div_A3_A1_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] / train_metadata['A1_max_min_diff_flux']
train_metadata['div_A3_A0_max_min_flux'] = train_metadata['A3_max_min_diff_flux'] / train_metadata['A0_max_min_diff_flux']
train_metadata['div_A2_A1_max_min_flux'] = train_metadata['A2_max_min_diff_flux'] / train_metadata['A1_max_min_diff_flux']
train_metadata['div_A2_A0_max_min_flux'] = train_metadata['A2_max_min_diff_flux'] / train_metadata['A0_max_min_diff_flux']
train_metadata['div_A1_A0_max_min_flux'] = train_metadata['A1_max_min_diff_flux'] / train_metadata['A0_max_min_diff_flux']
#F2
train_metadata['div_A5_A4_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] / train_metadata['A4_max_mean_diff_flux']
train_metadata['div_A5_A3_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] / train_metadata['A3_max_mean_diff_flux']
train_metadata['div_A5_A2_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] / train_metadata['A2_max_mean_diff_flux']
train_metadata['div_A5_A1_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] / train_metadata['A1_max_mean_diff_flux']
train_metadata['div_A5_A0_max_mean_flux'] = train_metadata['A5_max_mean_diff_flux'] / train_metadata['A0_max_mean_diff_flux']
train_metadata['div_A4_A3_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] / train_metadata['A3_max_mean_diff_flux']
train_metadata['div_A4_A2_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] / train_metadata['A2_max_mean_diff_flux']
train_metadata['div_A4_A1_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] / train_metadata['A1_max_mean_diff_flux']
train_metadata['div_A4_A0_max_mean_flux'] = train_metadata['A4_max_mean_diff_flux'] / train_metadata['A0_max_mean_diff_flux']
train_metadata['div_A3_A2_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] / train_metadata['A2_max_mean_diff_flux']
train_metadata['div_A3_A1_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] / train_metadata['A1_max_mean_diff_flux']
train_metadata['div_A3_A0_max_mean_flux'] = train_metadata['A3_max_mean_diff_flux'] / train_metadata['A0_max_mean_diff_flux']
train_metadata['div_A2_A1_max_mean_flux'] = train_metadata['A2_max_mean_diff_flux'] / train_metadata['A1_max_mean_diff_flux']
train_metadata['div_A2_A0_max_mean_flux'] = train_metadata['A2_max_mean_diff_flux'] / train_metadata['A0_max_mean_diff_flux']
train_metadata['div_A1_A0_max_mean_flux'] = train_metadata['A1_max_mean_diff_flux'] / train_metadata['A0_max_mean_diff_flux']
#F3
train_metadata['div_A5_A4_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] / train_metadata['A4_max_median_diff_flux']
train_metadata['div_A5_A3_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] / train_metadata['A3_max_median_diff_flux']
train_metadata['div_A5_A2_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] / train_metadata['A2_max_median_diff_flux']
train_metadata['div_A5_A1_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] / train_metadata['A1_max_median_diff_flux']
train_metadata['div_A5_A0_max_median_flux'] = train_metadata['A5_max_median_diff_flux'] / train_metadata['A0_max_median_diff_flux']
train_metadata['div_A4_A3_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] / train_metadata['A3_max_median_diff_flux']
train_metadata['div_A4_A2_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] / train_metadata['A2_max_median_diff_flux']
train_metadata['div_A4_A1_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] / train_metadata['A1_max_median_diff_flux']
train_metadata['div_A4_A0_max_median_flux'] = train_metadata['A4_max_median_diff_flux'] / train_metadata['A0_max_median_diff_flux']
train_metadata['div_A3_A2_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] / train_metadata['A2_max_median_diff_flux']
train_metadata['div_A3_A1_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] / train_metadata['A1_max_median_diff_flux']
train_metadata['div_A3_A0_max_median_flux'] = train_metadata['A3_max_median_diff_flux'] / train_metadata['A0_max_median_diff_flux']
train_metadata['div_A2_A1_max_median_flux'] = train_metadata['A2_max_median_diff_flux'] / train_metadata['A1_max_median_diff_flux']
train_metadata['div_A2_A0_max_median_flux'] = train_metadata['A2_max_median_diff_flux'] / train_metadata['A0_max_median_diff_flux']
train_metadata['div_A1_A0_max_median_flux'] = train_metadata['A1_max_median_diff_flux'] / train_metadata['A0_max_median_diff_flux']
#F4
train_metadata['div_A5_A4_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] / train_metadata['A4_median_mean_diff_flux']
train_metadata['div_A5_A3_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] / train_metadata['A3_median_mean_diff_flux']
train_metadata['div_A5_A2_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] / train_metadata['A2_median_mean_diff_flux']
train_metadata['div_A5_A1_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] / train_metadata['A1_median_mean_diff_flux']
train_metadata['div_A5_A0_median_mean_flux'] = train_metadata['A5_median_mean_diff_flux'] / train_metadata['A0_median_mean_diff_flux']
train_metadata['div_A4_A3_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] / train_metadata['A3_median_mean_diff_flux']
train_metadata['div_A4_A2_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] / train_metadata['A2_median_mean_diff_flux']
train_metadata['div_A4_A1_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] / train_metadata['A1_median_mean_diff_flux']
train_metadata['div_A4_A0_median_mean_flux'] = train_metadata['A4_median_mean_diff_flux'] / train_metadata['A0_median_mean_diff_flux']
train_metadata['div_A3_A2_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] / train_metadata['A2_median_mean_diff_flux']
train_metadata['div_A3_A1_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] / train_metadata['A1_median_mean_diff_flux']
train_metadata['div_A3_A0_median_mean_flux'] = train_metadata['A3_median_mean_diff_flux'] / train_metadata['A0_median_mean_diff_flux']
train_metadata['div_A2_A1_median_mean_flux'] = train_metadata['A2_median_mean_diff_flux'] / train_metadata['A1_median_mean_diff_flux']
train_metadata['div_A2_A0_median_mean_flux'] = train_metadata['A2_median_mean_diff_flux'] / train_metadata['A0_median_mean_diff_flux']
train_metadata['div_A1_A0_median_mean_flux'] = train_metadata['A1_median_mean_diff_flux'] / train_metadata['A0_median_mean_diff_flux']
#F5
train_metadata['div_A5_A4_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] / train_metadata['A4_median_min_diff_flux']
train_metadata['div_A5_A3_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] / train_metadata['A3_median_min_diff_flux']
train_metadata['div_A5_A2_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] / train_metadata['A2_median_min_diff_flux']
train_metadata['div_A5_A1_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] / train_metadata['A1_median_min_diff_flux']
train_metadata['div_A5_A0_median_min_flux'] = train_metadata['A5_median_min_diff_flux'] / train_metadata['A0_median_min_diff_flux']
train_metadata['div_A4_A3_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] / train_metadata['A3_median_min_diff_flux']
train_metadata['div_A4_A2_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] / train_metadata['A2_median_min_diff_flux']
train_metadata['div_A4_A1_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] / train_metadata['A1_median_min_diff_flux']
train_metadata['div_A4_A0_median_min_flux'] = train_metadata['A4_median_min_diff_flux'] / train_metadata['A0_median_min_diff_flux']
train_metadata['div_A3_A2_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] / train_metadata['A2_median_min_diff_flux']
train_metadata['div_A3_A1_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] / train_metadata['A1_median_min_diff_flux']
train_metadata['div_A3_A0_median_min_flux'] = train_metadata['A3_median_min_diff_flux'] / train_metadata['A0_median_min_diff_flux']
train_metadata['div_A2_A1_median_min_flux'] = train_metadata['A2_median_min_diff_flux'] / train_metadata['A1_median_min_diff_flux']
train_metadata['div_A2_A0_median_min_flux'] = train_metadata['A2_median_min_diff_flux'] / train_metadata['A0_median_min_diff_flux']
train_metadata['div_A1_A0_median_min_flux'] = train_metadata['A1_median_min_diff_flux'] / train_metadata['A0_median_min_diff_flux']
#F6
train_metadata['div_A5_A4_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] / train_metadata['A4_mean_min_diff_flux']
train_metadata['div_A5_A3_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] / train_metadata['A3_mean_min_diff_flux']
train_metadata['div_A5_A2_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] / train_metadata['A2_mean_min_diff_flux']
train_metadata['div_A5_A1_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] / train_metadata['A1_mean_min_diff_flux']
train_metadata['div_A5_A0_mean_min_flux'] = train_metadata['A5_mean_min_diff_flux'] / train_metadata['A0_mean_min_diff_flux']
train_metadata['div_A4_A3_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] / train_metadata['A3_mean_min_diff_flux']
train_metadata['div_A4_A2_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] / train_metadata['A2_mean_min_diff_flux']
train_metadata['div_A4_A1_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] / train_metadata['A1_mean_min_diff_flux']
train_metadata['div_A4_A0_mean_min_flux'] = train_metadata['A4_mean_min_diff_flux'] / train_metadata['A0_mean_min_diff_flux']
train_metadata['div_A3_A2_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] / train_metadata['A2_mean_min_diff_flux']
train_metadata['div_A3_A1_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] / train_metadata['A1_mean_min_diff_flux']
train_metadata['div_A3_A0_mean_min_flux'] = train_metadata['A3_mean_min_diff_flux'] / train_metadata['A0_mean_min_diff_flux']
train_metadata['div_A2_A1_mean_min_flux'] = train_metadata['A2_mean_min_diff_flux'] / train_metadata['A1_mean_min_diff_flux']
train_metadata['div_A2_A0_mean_min_flux'] = train_metadata['A2_mean_min_diff_flux'] / train_metadata['A0_mean_min_diff_flux']
train_metadata['div_A1_A0_mean_min_flux'] = train_metadata['A1_mean_min_diff_flux'] / train_metadata['A0_mean_min_diff_flux']
#F7
train_metadata['div_A5_A4_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] / train_metadata['A4_plus_3_sigma']
train_metadata['div_A5_A3_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] / train_metadata['A3_plus_3_sigma']
train_metadata['div_A5_A2_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] / train_metadata['A2_plus_3_sigma']
train_metadata['div_A5_A1_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] / train_metadata['A1_plus_3_sigma']
train_metadata['div_A5_A0_plus_3_sigma'] = train_metadata['A5_plus_3_sigma'] / train_metadata['A0_plus_3_sigma']
train_metadata['div_A4_A3_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] / train_metadata['A3_plus_3_sigma']
train_metadata['div_A4_A2_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] / train_metadata['A2_plus_3_sigma']
train_metadata['div_A4_A1_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] / train_metadata['A1_plus_3_sigma']
train_metadata['div_A4_A0_plus_3_sigma'] = train_metadata['A4_plus_3_sigma'] / train_metadata['A0_plus_3_sigma']
train_metadata['div_A3_A2_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] / train_metadata['A2_plus_3_sigma']
train_metadata['div_A3_A1_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] / train_metadata['A1_plus_3_sigma']
train_metadata['div_A3_A0_plus_3_sigma'] = train_metadata['A3_plus_3_sigma'] / train_metadata['A0_plus_3_sigma']
train_metadata['div_A2_A1_plus_3_sigma'] = train_metadata['A2_plus_3_sigma'] / train_metadata['A1_plus_3_sigma']
train_metadata['div_A2_A0_plus_3_sigma'] = train_metadata['A2_plus_3_sigma'] / train_metadata['A0_plus_3_sigma']
train_metadata['div_A1_A0_plus_3_sigma'] = train_metadata['A1_plus_3_sigma'] / train_metadata['A0_plus_3_sigma']
#F8
train_metadata['div_A5_A4_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] / train_metadata['A4_plus_2_sigma']
train_metadata['div_A5_A3_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] / train_metadata['A3_plus_2_sigma']
train_metadata['div_A5_A2_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] / train_metadata['A2_plus_2_sigma']
train_metadata['div_A5_A1_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] / train_metadata['A1_plus_2_sigma']
train_metadata['div_A5_A0_plus_2_sigma'] = train_metadata['A5_plus_2_sigma'] / train_metadata['A0_plus_2_sigma']
train_metadata['div_A4_A3_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] / train_metadata['A3_plus_2_sigma']
train_metadata['div_A4_A2_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] / train_metadata['A2_plus_2_sigma']
train_metadata['div_A4_A1_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] / train_metadata['A1_plus_2_sigma']
train_metadata['div_A4_A0_plus_2_sigma'] = train_metadata['A4_plus_2_sigma'] / train_metadata['A0_plus_2_sigma']
train_metadata['div_A3_A2_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] / train_metadata['A2_plus_2_sigma']
train_metadata['div_A3_A1_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] / train_metadata['A1_plus_2_sigma']
train_metadata['div_A3_A0_plus_2_sigma'] = train_metadata['A3_plus_2_sigma'] / train_metadata['A0_plus_2_sigma']
train_metadata['div_A2_A1_plus_2_sigma'] = train_metadata['A2_plus_2_sigma'] / train_metadata['A1_plus_2_sigma']
train_metadata['div_A2_A0_plus_2_sigma'] = train_metadata['A2_plus_2_sigma'] / train_metadata['A0_plus_2_sigma']
train_metadata['div_A1_A0_plus_2_sigma'] = train_metadata['A1_plus_2_sigma'] / train_metadata['A0_plus_2_sigma']
#F9
train_metadata['div_A5_A4_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] / train_metadata['A4_plus_1_sigma']
train_metadata['div_A5_A3_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] / train_metadata['A3_plus_1_sigma']
train_metadata['div_A5_A2_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] / train_metadata['A2_plus_1_sigma']
train_metadata['div_A5_A1_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] / train_metadata['A1_plus_1_sigma']
train_metadata['div_A5_A0_plus_1_sigma'] = train_metadata['A5_plus_1_sigma'] / train_metadata['A0_plus_1_sigma']
train_metadata['div_A4_A3_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] / train_metadata['A3_plus_1_sigma']
train_metadata['div_A4_A2_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] / train_metadata['A2_plus_1_sigma']
train_metadata['div_A4_A1_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] / train_metadata['A1_plus_1_sigma']
train_metadata['div_A4_A0_plus_1_sigma'] = train_metadata['A4_plus_1_sigma'] / train_metadata['A0_plus_1_sigma']
train_metadata['div_A3_A2_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] / train_metadata['A2_plus_1_sigma']
train_metadata['div_A3_A1_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] / train_metadata['A1_plus_1_sigma']
train_metadata['div_A3_A0_plus_1_sigma'] = train_metadata['A3_plus_1_sigma'] / train_metadata['A0_plus_1_sigma']
train_metadata['div_A2_A1_plus_1_sigma'] = train_metadata['A2_plus_1_sigma'] / train_metadata['A1_plus_1_sigma']
train_metadata['div_A2_A0_plus_1_sigma'] = train_metadata['A2_plus_1_sigma'] / train_metadata['A0_plus_1_sigma']
train_metadata['div_A1_A0_plus_1_sigma'] = train_metadata['A1_plus_1_sigma'] / train_metadata['A0_plus_1_sigma']
#F10
train_metadata['div_A5_A4_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] / train_metadata['A4_minus_3_sigma']
train_metadata['div_A5_A3_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] / train_metadata['A3_minus_3_sigma']
train_metadata['div_A5_A2_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] / train_metadata['A2_minus_3_sigma']
train_metadata['div_A5_A1_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] / train_metadata['A1_minus_3_sigma']
train_metadata['div_A5_A0_minus_3_sigma'] = train_metadata['A5_minus_3_sigma'] / train_metadata['A0_minus_3_sigma']
train_metadata['div_A4_A3_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] / train_metadata['A3_minus_3_sigma']
train_metadata['div_A4_A2_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] / train_metadata['A2_minus_3_sigma']
train_metadata['div_A4_A1_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] / train_metadata['A1_minus_3_sigma']
train_metadata['div_A4_A0_minus_3_sigma'] = train_metadata['A4_minus_3_sigma'] / train_metadata['A0_minus_3_sigma']
train_metadata['div_A3_A2_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] / train_metadata['A2_minus_3_sigma']
train_metadata['div_A3_A1_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] / train_metadata['A1_minus_3_sigma']
train_metadata['div_A3_A0_minus_3_sigma'] = train_metadata['A3_minus_3_sigma'] / train_metadata['A0_minus_3_sigma']
train_metadata['div_A2_A1_minus_3_sigma'] = train_metadata['A2_minus_3_sigma'] / train_metadata['A1_minus_3_sigma']
train_metadata['div_A2_A0_minus_3_sigma'] = train_metadata['A2_minus_3_sigma'] / train_metadata['A0_minus_3_sigma']
train_metadata['div_A1_A0_minus_3_sigma'] = train_metadata['A1_minus_3_sigma'] / train_metadata['A0_minus_3_sigma']
#F11
train_metadata['div_A5_A4_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] / train_metadata['A4_minus_2_sigma']
train_metadata['div_A5_A3_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] / train_metadata['A3_minus_2_sigma']
train_metadata['div_A5_A2_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] / train_metadata['A2_minus_2_sigma']
train_metadata['div_A5_A1_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] / train_metadata['A1_minus_2_sigma']
train_metadata['div_A5_A0_minus_2_sigma'] = train_metadata['A5_minus_2_sigma'] / train_metadata['A0_minus_2_sigma']
train_metadata['div_A4_A3_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] / train_metadata['A3_minus_2_sigma']
train_metadata['div_A4_A2_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] / train_metadata['A2_minus_2_sigma']
train_metadata['div_A4_A1_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] / train_metadata['A1_minus_2_sigma']
train_metadata['div_A4_A0_minus_2_sigma'] = train_metadata['A4_minus_2_sigma'] / train_metadata['A0_minus_2_sigma']
train_metadata['div_A3_A2_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] / train_metadata['A2_minus_2_sigma']
train_metadata['div_A3_A1_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] / train_metadata['A1_minus_2_sigma']
train_metadata['div_A3_A0_minus_2_sigma'] = train_metadata['A3_minus_2_sigma'] / train_metadata['A0_minus_2_sigma']
train_metadata['div_A2_A1_minus_2_sigma'] = train_metadata['A2_minus_2_sigma'] / train_metadata['A1_minus_2_sigma']
train_metadata['div_A2_A0_minus_2_sigma'] = train_metadata['A2_minus_2_sigma'] / train_metadata['A0_minus_2_sigma']
train_metadata['div_A1_A0_minus_2_sigma'] = train_metadata['A1_minus_2_sigma'] / train_metadata['A0_minus_2_sigma']
#F12
train_metadata['div_A5_A4_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] / train_metadata['A4_minus_1_sigma']
train_metadata['div_A5_A3_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] / train_metadata['A3_minus_1_sigma']
train_metadata['div_A5_A2_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] / train_metadata['A2_minus_1_sigma']
train_metadata['div_A5_A1_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] / train_metadata['A1_minus_1_sigma']
train_metadata['div_A5_A0_minus_1_sigma'] = train_metadata['A5_minus_1_sigma'] / train_metadata['A0_minus_1_sigma']
train_metadata['div_A4_A3_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] / train_metadata['A3_minus_1_sigma']
train_metadata['div_A4_A2_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] / train_metadata['A2_minus_1_sigma']
train_metadata['div_A4_A1_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] / train_metadata['A1_minus_1_sigma']
train_metadata['div_A4_A0_minus_1_sigma'] = train_metadata['A4_minus_1_sigma'] / train_metadata['A0_minus_1_sigma']
train_metadata['div_A3_A2_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] / train_metadata['A2_minus_1_sigma']
train_metadata['div_A3_A1_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] / train_metadata['A1_minus_1_sigma']
train_metadata['div_A3_A0_minus_1_sigma'] = train_metadata['A3_minus_1_sigma'] / train_metadata['A0_minus_1_sigma']
train_metadata['div_A2_A1_minus_1_sigma'] = train_metadata['A2_minus_1_sigma'] / train_metadata['A1_minus_1_sigma']
train_metadata['div_A2_A0_minus_1_sigma'] = train_metadata['A2_minus_1_sigma'] / train_metadata['A0_minus_1_sigma']
train_metadata['div_A1_A0_minus_1_sigma'] = train_metadata['A1_minus_1_sigma'] / train_metadata['A0_minus_1_sigma']

In [20]:
print(train_metadata.shape)

(7848, 283)


In [41]:
train_metadata.iloc[:,103:].head()

,div_A5_A4_max_min_flux,div_A5_A3_max_min_flux,div_A5_A2_max_min_flux,div_A5_A1_max_min_flux,div_A5_A0_max_min_flux,div_A4_A3_max_min_flux,div_A4_A2_max_min_flux,div_A4_A1_max_min_flux,div_A4_A0_max_min_flux,div_A3_A2_max_min_flux,div_A3_A1_max_min_flux,div_A3_A0_max_min_flux,div_A2_A1_max_min_flux,div_A2_A0_max_min_flux,div_A1_A0_max_min_flux,div_A5_A4_max_mean_flux,div_A5_A3_max_mean_flux,div_A5_A2_max_mean_flux,div_A5_A1_max_mean_flux,div_A5_A0_max_mean_flux,div_A4_A3_max_mean_flux,div_A4_A2_max_mean_flux,div_A4_A1_max_mean_flux,div_A4_A0_max_mean_flux,div_A3_A2_max_mean_flux,div_A3_A1_max_mean_flux,div_A3_A0_max_mean_flux,div_A2_A1_max_mean_flux,div_A2_A0_max_mean_flux,div_A1_A0_max_mean_flux,div_A5_A4_max_median_flux,div_A5_A3_max_median_flux,div_A5_A2_max_median_flux,div_A5_A1_max_median_flux,div_A5_A0_max_median_flux,div_A4_A3_max_median_flux,div_A4_A2_max_median_flux,div_A4_A1_max_median_flux,div_A4_A0_max_median_flux,div_A3_A2_max_median_flux,div_A3_A1_max_median_flux,div_A3_A0_max_median_flux,div_A2_A1_max_median_flux,div_A2_A0_max_median_flux,div_A1_A0_max_median_flux,div_A5_A4_median_mean_flux,div_A5_A3_median_mean_flux,div_A5_A2_median_mean_flux,div_A5_A1_median_mean_flux,div_A5_A0_median_mean_flux,div_A4_A3_median_mean_flux,div_A4_A2_median_mean_flux,div_A4_A1_median_mean_flux,div_A4_A0_median_mean_flux,div_A3_A2_median_mean_flux,div_A3_A1_median_mean_flux,div_A3_A0_median_mean_flux,div_A2_A1_median_mean_flux,div_A2_A0_median_mean_flux,div_A1_A0_median_mean_flux,div_A5_A4_median_min_flux,div_A5_A3_median_min_flux,div_A5_A2_median_min_flux,div_A5_A1_median_min_flux,div_A5_A0_median_min_flux,div_A4_A3_median_min_flux,div_A4_A2_median_min_flux,div_A4_A1_median_min_flux,div_A4_A0_median_min_flux,div_A3_A2_median_min_flux,div_A3_A1_median_min_flux,div_A3_A0_median_min_flux,div_A2_A1_median_min_flux,div_A2_A0_median_min_flux,div_A1_A0_median_min_flux,div_A5_A4_mean_min_flux,div_A5_A3_mean_min_flux,div_A5_A2_mean_min_flux,div_A5_A1_mean_min_flux,div_A5_A0_mean_min_flux,div_A4_A3_mean_min_flux,div_A4_A2_mean_min_flux,div_A4_A1_mean_min_flux,div_A4_A0_mean_min_flux,div_A3_A2_mean_min_flux,div_A3_A1_mean_min_flux,div_A3_A0_mean_min_flux,div_A2_A1_mean_min_flux,div_A2_A0_mean_min_flux,div_A1_A0_mean_min_flux,div_A5_A4_plus_3_sigma,div_A5_A3_plus_3_sigma,div_A5_A2_plus_3_sigma,div_A5_A1_plus_3_sigma,div_A5_A0_plus_3_sigma,div_A4_A3_plus_3_sigma,div_A4_A2_plus_3_sigma,div_A4_A1_plus_3_sigma,div_A4_A0_plus_3_sigma,div_A3_A2_plus_3_sigma,div_A3_A1_plus_3_sigma,div_A3_A0_plus_3_sigma,div_A2_A1_plus_3_sigma,div_A2_A0_plus_3_sigma,div_A1_A0_plus_3_sigma,div_A5_A4_plus_2_sigma,div_A5_A3_plus_2_sigma,div_A5_A2_plus_2_sigma,div_A5_A1_plus_2_sigma,div_A5_A0_plus_2_sigma,div_A4_A3_plus_2_sigma,div_A4_A2_plus_2_sigma,div_A4_A1_plus_2_sigma,div_A4_A0_plus_2_sigma,div_A3_A2_plus_2_sigma,div_A3_A1_plus_2_sigma,div_A3_A0_plus_2_sigma,div_A2_A1_plus_2_sigma,div_A2_A0_plus_2_sigma,div_A1_A0_plus_2_sigma,div_A5_A4_plus_1_sigma,div_A5_A3_plus_1_sigma,div_A5_A2_plus_1_sigma,div_A5_A1_plus_1_sigma,div_A5_A0_plus_1_sigma,div_A4_A3_plus_1_sigma,div_A4_A2_plus_1_sigma,div_A4_A1_plus_1_sigma,div_A4_A0_plus_1_sigma,div_A3_A2_plus_1_sigma,div_A3_A1_plus_1_sigma,div_A3_A0_plus_1_sigma,div_A2_A1_plus_1_sigma,div_A2_A0_plus_1_sigma,div_A1_A0_plus_1_sigma,div_A5_A4_minus_3_sigma,div_A5_A3_minus_3_sigma,div_A5_A2_minus_3_sigma,div_A5_A1_minus_3_sigma,div_A5_A0_minus_3_sigma,div_A4_A3_minus_3_sigma,div_A4_A2_minus_3_sigma,div_A4_A1_minus_3_sigma,div_A4_A0_minus_3_sigma,div_A3_A2_minus_3_sigma,div_A3_A1_minus_3_sigma,div_A3_A0_minus_3_sigma,div_A2_A1_minus_3_sigma,div_A2_A0_minus_3_sigma,div_A1_A0_minus_3_sigma,div_A5_A4_minus_2_sigma,div_A5_A3_minus_2_sigma,div_A5_A2_minus_2_sigma,div_A5_A1_minus_2_sigma,div_A5_A0_minus_2_sigma,div_A4_A3_minus_2_sigma,div_A4_A2_minus_2_sigma,div_A4_A1_minus_2_sigma,div_A4_A0_minus_2_sigma,div_A3_A2_minus_2_sigma,div_A3_A1_minus_2_sigma,div_A3_A0_minus_2_sigma,div_A2_A1_minus_2_sigma,div_A2_A0_minus_2_sigma,div_A1_A0_minus_2_

In [42]:
print(train_metadata_kaggle.shape)

(7848, 133)


In [26]:
test_id = test_metadata_kaggle['object_id']

In [27]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [89]:
"""used_columns1 = ['div_A4_A2_median_min_flux', 'div_A5_A2_median_min_flux','div_A5_A2_minus_1_sigma',
                'div_A5_A4_median_mean_flux','div_A3_A0_plus_1_sigma','div_A4_A1_median_min_flux',
                'div_A4_A1_minus_1_sigma','div_A1_A0_median_min_flux','div_A5_A0_max_median_flux',
                'div_A4_A0_minus_1_sigma']"""

In [137]:
used_columns1 = ['div_A4_A2_median_min_flux', 'div_A5_A2_median_min_flux','div_A5_A2_minus_1_sigma',
                'div_A5_A4_median_mean_flux','div_A3_A0_plus_1_sigma','div_A4_A1_minus_1_sigma']

In [138]:
"""used_columns2 = ['div_A5_A1_median_min_flux','div_A4_A3_max_mean_flux',
                'div_A5_A0_minus_1_sigma','div_A3_A1_max_median_flux',
                'div_A5_A1_median_mean_flux','div_A2_A1_minus_1_sigma',
                'div_A4_A2_max_mean_flux','div_A3_A2_median_min_flux',
                'div_A5_A0_plus_1_sigma']"""

"used_columns2 = ['div_A5_A1_median_min_flux','div_A4_A3_max_mean_flux',\n                'div_A5_A0_minus_1_sigma','div_A3_A1_max_median_flux',\n                'div_A5_A1_median_mean_flux','div_A2_A1_minus_1_sigma',\n                'div_A4_A2_max_mean_flux','div_A3_A2_median_min_flux',\n                'div_A5_A0_plus_1_sigma']"

In [139]:
train_metadata[list(train_metadata.columns)[-180:]].head()

,div_A5_A4_max_min_flux,div_A5_A3_max_min_flux,div_A5_A2_max_min_flux,div_A5_A1_max_min_flux,div_A5_A0_max_min_flux,div_A4_A3_max_min_flux,div_A4_A2_max_min_flux,div_A4_A1_max_min_flux,div_A4_A0_max_min_flux,div_A3_A2_max_min_flux,div_A3_A1_max_min_flux,div_A3_A0_max_min_flux,div_A2_A1_max_min_flux,div_A2_A0_max_min_flux,div_A1_A0_max_min_flux,div_A5_A4_max_mean_flux,div_A5_A3_max_mean_flux,div_A5_A2_max_mean_flux,div_A5_A1_max_mean_flux,div_A5_A0_max_mean_flux,div_A4_A3_max_mean_flux,div_A4_A2_max_mean_flux,div_A4_A1_max_mean_flux,div_A4_A0_max_mean_flux,div_A3_A2_max_mean_flux,div_A3_A1_max_mean_flux,div_A3_A0_max_mean_flux,div_A2_A1_max_mean_flux,div_A2_A0_max_mean_flux,div_A1_A0_max_mean_flux,div_A5_A4_max_median_flux,div_A5_A3_max_median_flux,div_A5_A2_max_median_flux,div_A5_A1_max_median_flux,div_A5_A0_max_median_flux,div_A4_A3_max_median_flux,div_A4_A2_max_median_flux,div_A4_A1_max_median_flux,div_A4_A0_max_median_flux,div_A3_A2_max_median_flux,div_A3_A1_max_median_flux,div_A3_A0_max_median_flux,div_A2_A1_max_median_flux,div_A2_A0_max_median_flux,div_A1_A0_max_median_flux,div_A5_A4_median_mean_flux,div_A5_A3_median_mean_flux,div_A5_A2_median_mean_flux,div_A5_A1_median_mean_flux,div_A5_A0_median_mean_flux,div_A4_A3_median_mean_flux,div_A4_A2_median_mean_flux,div_A4_A1_median_mean_flux,div_A4_A0_median_mean_flux,div_A3_A2_median_mean_flux,div_A3_A1_median_mean_flux,div_A3_A0_median_mean_flux,div_A2_A1_median_mean_flux,div_A2_A0_median_mean_flux,div_A1_A0_median_mean_flux,div_A5_A4_median_min_flux,div_A5_A3_median_min_flux,div_A5_A2_median_min_flux,div_A5_A1_median_min_flux,div_A5_A0_median_min_flux,div_A4_A3_median_min_flux,div_A4_A2_median_min_flux,div_A4_A1_median_min_flux,div_A4_A0_median_min_flux,div_A3_A2_median_min_flux,div_A3_A1_median_min_flux,div_A3_A0_median_min_flux,div_A2_A1_median_min_flux,div_A2_A0_median_min_flux,div_A1_A0_median_min_flux,div_A5_A4_mean_min_flux,div_A5_A3_mean_min_flux,div_A5_A2_mean_min_flux,div_A5_A1_mean_min_flux,div_A5_A0_mean_min_flux,div_A4_A3_mean_min_flux,div_A4_A2_mean_min_flux,div_A4_A1_mean_min_flux,div_A4_A0_mean_min_flux,div_A3_A2_mean_min_flux,div_A3_A1_mean_min_flux,div_A3_A0_mean_min_flux,div_A2_A1_mean_min_flux,div_A2_A0_mean_min_flux,div_A1_A0_mean_min_flux,div_A5_A4_plus_3_sigma,div_A5_A3_plus_3_sigma,div_A5_A2_plus_3_sigma,div_A5_A1_plus_3_sigma,div_A5_A0_plus_3_sigma,div_A4_A3_plus_3_sigma,div_A4_A2_plus_3_sigma,div_A4_A1_plus_3_sigma,div_A4_A0_plus_3_sigma,div_A3_A2_plus_3_sigma,div_A3_A1_plus_3_sigma,div_A3_A0_plus_3_sigma,div_A2_A1_plus_3_sigma,div_A2_A0_plus_3_sigma,div_A1_A0_plus_3_sigma,div_A5_A4_plus_2_sigma,div_A5_A3_plus_2_sigma,div_A5_A2_plus_2_sigma,div_A5_A1_plus_2_sigma,div_A5_A0_plus_2_sigma,div_A4_A3_plus_2_sigma,div_A4_A2_plus_2_sigma,div_A4_A1_plus_2_sigma,div_A4_A0_plus_2_sigma,div_A3_A2_plus_2_sigma,div_A3_A1_plus_2_sigma,div_A3_A0_plus_2_sigma,div_A2_A1_plus_2_sigma,div_A2_A0_plus_2_sigma,div_A1_A0_plus_2_sigma,div_A5_A4_plus_1_sigma,div_A5_A3_plus_1_sigma,div_A5_A2_plus_1_sigma,div_A5_A1_plus_1_sigma,div_A5_A0_plus_1_sigma,div_A4_A3_plus_1_sigma,div_A4_A2_plus_1_sigma,div_A4_A1_plus_1_sigma,div_A4_A0_plus_1_sigma,div_A3_A2_plus_1_sigma,div_A3_A1_plus_1_sigma,div_A3_A0_plus_1_sigma,div_A2_A1_plus_1_sigma,div_A2_A0_plus_1_sigma,div_A1_A0_plus_1_sigma,div_A5_A4_minus_3_sigma,div_A5_A3_minus_3_sigma,div_A5_A2_minus_3_sigma,div_A5_A1_minus_3_sigma,div_A5_A0_minus_3_sigma,div_A4_A3_minus_3_sigma,div_A4_A2_minus_3_sigma,div_A4_A1_minus_3_sigma,div_A4_A0_minus_3_sigma,div_A3_A2_minus_3_sigma,div_A3_A1_minus_3_sigma,div_A3_A0_minus_3_sigma,div_A2_A1_minus_3_sigma,div_A2_A0_minus_3_sigma,div_A1_A0_minus_3_sigma,div_A5_A4_minus_2_sigma,div_A5_A3_minus_2_sigma,div_A5_A2_minus_2_sigma,div_A5_A1_minus_2_sigma,div_A5_A0_minus_2_sigma,div_A4_A3_minus_2_sigma,div_A4_A2_minus_2_sigma,div_A4_A1_minus_2_sigma,div_A4_A0_minus_2_sigma,div_A3_A2_minus_2_sigma,div_A3_A1_minus_2_sigma,div_A3_A0_minus_2_sigma,div_A2_A1_minus_2_sigma,div_A2_A0_minus_2_sigma,div_A1_A0_minus_2_

In [140]:
#used_column4 = ['diff_A5_A2_median_min_flux','diff_A1_A0_minus_1_sigma','diff_A5_A1_max_min_flux']

In [141]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + used_columns1],on = 'object_id',how = 'left')
print(temp.shape)
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 1000, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10,
'min_child_weight': 200.0, 
#'min_child_weight': 100.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
#'reg_alpha': 0.1,
'reg_alpha': 0.0, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 139)
0 0.5846232690590423
1 0.5598478406280183
2 0.6060594241195905
3 0.5551329424743721
4 0.5738283897556737
MULTI WEIGHTED LOG LOSS : 0.57588 
CPU times: user 11min 16s, sys: 1.19 s, total: 11min 17s
Wall time: 1min 30s


In [33]:
imp_df.sort_values(by = 'gain',ascending=False)

,feature,gain,fold
57,distmod,1048,5
55,hostgal_photoz,1005,5
54,mjd_diff_det,921,5
61,hostgal_photoz_certain,565,5
56,hostgal_photoz_err,534,5
52,mjd__mean_abs_change,423,5
3,flux_median,389,5
46,flux__longest_strike_above_mean,370,5
75,__median_absolute_deviation___2_,358,5
19,flux_diff2,357,5


In [ ]:
"""%%time
final_dict = {}

for column_ in list(train_metadata.columns)[233:]:
    loss_list = []
    temp = train_metadata_kaggle.copy()
    temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
    y = temp['target']
    del temp['target']
    classes = sorted(y.unique())

    class_weight = {
        c: 1 for c in classes
    }
    for c in [64, 15]:
        class_weight[c] = 2

    train_id = temp['object_id']
    del temp['object_id']
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
    'random_state':51,
    'device': 'cpu', 
    'objective': 'multiclass', 
    'num_class': 14, 
    'boosting_type': 'gbdt', 
    'n_jobs': -1, 
    'max_depth': 7, 
    'n_estimators': 1000, 
    'subsample_freq': 2, 
    'subsample_for_bin': 5000, 
    'min_data_per_group': 100, 
    'max_cat_to_onehot': 4, 
    'cat_l2': 1.0, 
    'cat_smooth': 59.5, 
    'max_cat_threshold': 32, 
    'metric_freq': 10, 
    'verbosity': -1, 
    'metric': 'multi_logloss', 
    'xgboost_dart_mode': False, 
    'uniform_drop': False, 
    'colsample_bytree': 0.5, 
    'drop_rate': 0.173, 
    'learning_rate': 0.0267, 
    'max_drop': 5, 
    'min_child_samples': 10,
    'min_child_weight': 200.0, 
    #'min_child_weight': 100.0, 
    'min_split_gain': 0.1, 
    'num_leaves': 7, 
    #'reg_alpha': 0.1,
    'reg_alpha': 0.0, 
    'reg_lambda': 0.00023, 
    'skip_drop': 0.44, 
    'subsample': 0.75}
    oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
        val_x, val_y = temp.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=False,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
        loss_list.append(loss_oof)
        #print(fold_,loss_oof)

        imp_df = pd.DataFrame()
        imp_df['feature'] = temp.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(column_,'MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
    final_dict[column_] = loss_list"""

In [ ]:
"""
final_dict2 = pd.Series(final_dict)
final_dict2 = pd.DataFrame(final_dict2)
final_dict2['fold1'] = final_dict2[0].apply(lambda x: x[0])
final_dict2['fold2'] = final_dict2[0].apply(lambda x: x[1])
final_dict2['fold3'] = final_dict2[0].apply(lambda x: x[2])
final_dict2['fold4'] = final_dict2[0].apply(lambda x: x[3])
final_dict2['fold5'] = final_dict2[0].apply(lambda x: x[4])

del final_dict2[0]

final_dict2 = final_dict2.reset_index(drop=False)

final_dict2 = final_dict2.rename(columns={'index':'column_name'})

final_dict2['fold1_1'] = (final_dict2['fold1'] > 0.62756) * 1
final_dict2['fold2_1'] = (final_dict2['fold2'] > 0.59905) * 1
final_dict2['fold3_1'] = (final_dict2['fold3'] > 0.65624) * 1
final_dict2['fold4_1'] = (final_dict2['fold4'] > 0.60978) * 1
final_dict2['fold5_1'] = (final_dict2['fold5'] > 0.61012) * 1
final_dict2['fold_sum'] = final_dict2['fold1_1']+ final_dict2['fold2_1'] + final_dict2['fold3_1'] + final_dict2['fold4_1'] + final_dict2['fold5_1']
"""

In [ ]:
#final_dict2[final_dict2['column_name'].isin(most_imp_ones)]
#final_dict2[final_dict2['fold_sum'] == 1]

In [ ]:
#final_dict2.head()

In [ ]:
#final_dict2.to_csv('final_features3.csv',index=False)

In [143]:
#modify to work with kfold
#def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):
    
        
    sm=SMOTE(random_state=51)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

        
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [145]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()

#temp = temp.merge(train_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')

temp = temp.merge(train_metadata[['object_id'] + used_columns1  ],on = 'object_id',how = 'left')


print(temp.shape)
temp.fillna(0, inplace=True)

y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 1000, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10, 
'min_child_weight': 200.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
'reg_alpha': 0.0, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    trn_xa, trn_y, val_xa, val_y=smoteAdataset(trn_x.values, trn_y.values, val_x.values, val_y.values)
    trn_x=pd.DataFrame(data=trn_xa, columns=trn_x.columns)
    val_x=pd.DataFrame(data=val_xa, columns=val_x.columns)
    
    print(trn_x.shape,trn_y.shape,val_x.shape,val_y.shape)
    
    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 139)
(25900, 137) (25900,) (1574, 137) (1574,)
0 0.5307443319232668
(25900, 137) (25900,) (1572, 137) (1572,)
1 0.5564314031865383
(25900, 137) (25900,) (1571, 137) (1571,)
2 0.5893145525114981
(25914, 137) (25914,) (1567, 137) (1567,)
3 0.5409958222769201
(25914, 137) (25914,) (1564, 137) (1564,)
4 0.5588997848985131
MULTI WEIGHTED LOG LOSS : 0.55516 
CPU times: user 28min 25s, sys: 2.4 s, total: 28min 27s
Wall time: 4min 3s


In [ ]:
#used_columns = used_columns1 + used_columns2 + used_columns3 

In [147]:
used_columns1

['div_A4_A2_median_min_flux',
 'div_A5_A2_median_min_flux',
 'div_A5_A2_minus_1_sigma',
 'div_A5_A4_median_mean_flux',
 'div_A3_A0_plus_1_sigma',
 'div_A4_A1_minus_1_sigma']

In [148]:
test_metadata['div_A4_A2_median_min_flux'] = test_metadata['A4_median_min_diff_flux'] / test_metadata['A2_median_min_diff_flux']
test_metadata['div_A5_A2_median_min_flux'] = test_metadata['A5_median_min_diff_flux'] / test_metadata['A2_median_min_diff_flux']
test_metadata['div_A5_A2_minus_1_sigma'] = test_metadata['A5_minus_1_sigma'] / test_metadata['A2_minus_1_sigma']
test_metadata['div_A5_A4_median_mean_flux'] = test_metadata['A5_median_mean_diff_flux'] / test_metadata['A4_median_mean_diff_flux']
test_metadata['div_A3_A0_plus_1_sigma'] = test_metadata['A3_plus_1_sigma'] / test_metadata['A0_plus_1_sigma']
test_metadata['div_A4_A1_minus_1_sigma'] = test_metadata['A4_minus_1_sigma'] / test_metadata['A1_minus_1_sigma']

In [149]:
print(train_metadata.shape,test_metadata.shape)

(7848, 283) (3492890, 109)


In [150]:
#train_metadata.drop([x for x in train_metadata.columns if x not in ['object_id'] + used_columns ] ,axis = 1,inplace=True)

In [151]:
#test_metadata = test_metadata[[x for x in test_metadata.columns if x in ['object_id'] + used_columns ]]

In [161]:
train_metadata_kaggle = train_metadata_kaggle.merge(train_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')
test_metadata_kaggle = test_metadata_kaggle.merge(test_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')

In [162]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 139) (3492890, 138)


In [163]:
train_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux,div_A4_A2_median_min_flux,div_A5_A2_median_min_flux,div_A5_A2_minus_1_sigma,div_A5_A4_median_mean_flux,div_A3_A0_plus_1_sigma,div_A4_A1_minus_1_sigma
0,615,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835268,1.744747,1.623740,0.946023,2.929669e+06,0.812722,-9.601766e+08,-1.414322,-327.742307,1761.066406,-14.306331,-5.373326,205.036926,1628.427737,-1.475181,0.128917,22370.594834,2806.374162,-1.255123,0.415580,7780.500807,2805.598113,-1.409885,0.339918,7024.003068,2536.068846,-1.449858,0.293128,3245.366349,2741.539785,-1.548319,0.200096,2704.641265,2893.344217,-1.592820,0.125268,352.0,19.0,29.0,202.114067,1.999688,35.0,4.0,2.631898,2.631898,873.7903,0.0000,0.0000,NaN,0.017,92,0.319006,-1.528827,0.000000,-205.03693,-3.254554,0.3528,0.9653,57,0.9830,0.2854,0.9120,0.362,-682.000,52,5.562230e-26,6.719410e-20,368.12900,6.217890,1.091730,5.49891,1.053490,114.465000,368.12900,0.172414,0.401664,0.129578,9.16612,9.508750,7.344980,289.27700,0.110785,646.9220,0.000,1.000000,1.0,0,0,0,0,-2191.161900,463.712450,-931.788

In [164]:
test_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux,div_A4_A2_median_min_flux,div_A5_A2_median_min_flux,div_A5_A2_minus_1_sigma,div_A5_A4_median_mean_flux,div_A3_A0_plus_1_sigma,div_A4_A1_minus_1_sigma
0,13,-12.680235,42.765503,3.997127,0.616561,9.149645,2.037355,0.691634,11.257108,2.461810,1.972973,1.718101,1.826388,0.157576,7806.412424,4.771625,1.896346e+05,5.396523,24.292155,55.445738,13.871398,2.282455,29.002872,37.684425,-0.247160,0.147622,92.039690,79.990817,10.567412,3.117684,164.640622,139.733762,4.512783,2.171455,300.547278,246.788411,2.951479,1.806797,408.305525,349.008202,0.094963,1.153076,324.515880,251.116361,0.072799,0.804929,330.0,15.0,32.0,4.426774,0.017885,9.0,222.0,2.350061,2.350061,119.8531,0.3193,0.0542,41.1123,0.019,1.851382,3.049709,0.337084,29.002872,0.402818,0.0000,0.07690,11.0,0.3137,0.3254,0.11770,0.7114,-1.778855,6.0,0.005627,0.528087,1.384690,-0.007597,0.610190,4.81984,0.681284,1.783390,1.384690,0.711538,0.255030,0.090550,3.499440,2.788160,1.510790,13.24940,0.124063,13.15420,1.0,1.0,1.0,1,1,1,0,-13.032169,33.023636,-29.949324,30.439033,1.203725,-1.244573,3.146350,-9.275401,3.413229,-1.90

In [167]:
%%time
train_metadata_kaggle.to_csv('mydata_train_metadata.csv',index=False)
test_metadata_kaggle.to_csv('mydata_test_metadata.csv',index=False)

CPU times: user 9min 55s, sys: 8.56 s, total: 10min 4s
Wall time: 10min 4s


In [170]:
imp_df.sort_values('gain',ascending=False).head()

,feature,gain,fold
57,distmod,1592,5
55,hostgal_photoz,1557,5
54,mjd_diff_det,1386,5
61,hostgal_photoz_certain,873,5
56,hostgal_photoz_err,752,5


In [171]:
temp_test = test_metadata_kaggle.copy()

In [172]:
del temp_test['object_id']

In [173]:
temp_test.fillna(0,inplace = True)

In [174]:
print(temp.shape,temp_test.shape)

(7848, 137) (3492890, 137)


In [175]:
list(temp.columns) == list(temp_test.columns)

True

In [176]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

0
1
2
3
4
CPU times: user 4h 29min 16s, sys: 42.2 s, total: 4h 29min 59s
Wall time: 39min 38s


In [177]:
test_pred2 = pd.DataFrame()
test_pred2 = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [178]:
print(test_pred2.shape)

(3492890, 14)


In [179]:
#test_pred2 = pd.DataFrame(np.random.rand(10,14))

In [180]:
test_pred2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000179,0.001038,0.000141,0.805593,0.047541,0.000350,0.074855,0.000042,0.000131,0.002381,0.000263,0.066049,0.000270,0.001166
1,0.000121,0.006711,0.000132,0.136203,0.039605,0.000256,0.033613,0.000156,0.000490,0.018491,0.001953,0.750983,0.000250,0.011037
2,0.000263,0.016892,0.000293,0.139133,0.096388,0.000566,0.037680,0.004083,0.000686,0.061668,0.008164,0.603198,0.000369,0.030618
3,0.000313,0.002297,0.000565,0.052961,0.022658,0.000584,0.090238,0.003904,0.000502,0.578036,0.001415,0.195498,0.000462,0.050568
4,0.000071,0.001033,0.000079,0.047133,0.209952,0.000136,0.009144,0.000027,0.000092,0.004313,0.000096,0.727727,0.000104,0.000093


In [181]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [182]:
test_pred2.columns = temp_columns[1:15]

In [183]:
def getUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

In [184]:
klm = pd.DataFrame()
klm['mymean'] = test_pred2[feats].mean(axis=1)
klm['mymedian'] = test_pred2[feats].median(axis=1)
klm['mymax'] = test_pred2[feats].max(axis=1)

In [185]:
test_pred2['class_99'] = getUnknown(klm)

In [186]:
test_pred2.tail()

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
492885,0.000286,0.384119,0.000529,0.204638,0.119041,0.000606,0.121809,0.000283,0.002518,0.016498,0.002954,0.145506,0.000406,0.000808,0.245444
492886,0.000296,0.009702,0.000319,0.036255,0.213757,0.000397,0.017968,0.387354,0.000531,0.022403,0.001201,0.305817,0.000370,0.003629,0.246065
492887,0.000790,0.019711,0.000207,0.915776,0.023825,0.000475,0.027487,0.000072,0.000249,0.001283,0.000521,0.007552,0.000329,0.001722,0.067185
492888,0.000115,0.664491,0.000184,0.155340,0.004457,0.000217,0.011132,0.128702,0.000631,0.000390,0.000379,0.032990,0.000187,0.000785,0.185754
492889,0.000343,0.101076,0.000465,0.499759,0.140719,0.000773,0.060824,0.000149,0.000944,0.023887,0.001747,0.167210,0.001007,0.001097,0.228079


In [187]:
test_pred2 = test_pred2.reset_index(drop=True)

In [188]:
print(test_pred2.shape,test_id.shape)

(3492890, 15) (3492890,)


In [189]:
test_id.tail()

3492885    130787966
3492886    130787971
3492887    130787974
3492888    130788053
3492889    130788054
Name: object_id, dtype: int64

In [190]:
test_id = test_id.reset_index(drop=True)

In [191]:
test_id.index == test_pred2.index

array([ True,  True,  True, ...,  True,  True,  True])

In [192]:
%%time
test_pred = pd.concat([test_id,test_pred2],axis=1)

CPU times: user 87.9 ms, sys: 148 ms, total: 236 ms
Wall time: 234 ms


In [193]:
test_pred = test_pred[temp_columns]

In [194]:
test_pred.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.000179,0.001038,0.000141,0.805593,0.047541,0.000350,0.074855,0.000042,0.000131,0.002381,0.000263,0.066049,0.000270,0.001166,0.128398
1,14,0.000121,0.006711,0.000132,0.136203,0.039605,0.000256,0.033613,0.000156,0.000490,0.018491,0.001953,0.750983,0.000250,0.011037,0.154128
2,17,0.000263,0.016892,0.000293,0.139133,0.096388,0.000566,0.037680,0.004083,0.000686,0.061668,0.008164,0.603198,0.000369,0.030618,0.207193
3,23,0.000313,0.002297,0.000565,0.052961,0.022658,0.000584,0.090238,0.003904,0.000502,0.578036,0.001415,0.195498,0.000462,0.050568,0.211420
4,34,0.000071,0.001033,0.000079,0.047133,0.209952,0.000136,0.009144,0.000027,0.000092,0.004313,0.000096,0.727727,0.000104,0.000093,0.162610


In [195]:
print(test_pred.shape)

(3492890, 16)


In [196]:
%%time
test_pred.to_csv('test_pred_32.csv',index=False)

CPU times: user 1min 40s, sys: 1.95 s, total: 1min 42s
Wall time: 1min 42s


In [ ]:
#!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_27.csv -m "Message"